# Data Preperation


## Converting raw Rotten Tomatoes datasets into final dataset.

Import Libraries.

In [1]:
import pandas as pd

Merge the two datasets.

In [2]:
#combines the two datasets
def merge_datasets(df_movie, df_critic):

    #remove duplicate movies
    removed_duplicate_movies = df_movie.drop_duplicates(subset='rotten_tomatoes_link', keep='first')

    df_merged = pd.merge(df_critic,removed_duplicate_movies,on='rotten_tomatoes_link', how='inner')

    return df_merged

Clean the dataset.

In [3]:
#chooses columns we want and filters out anything we dont want
def fix_filter_dataset(df: pd.DataFrame) -> pd.DataFrame:

    wanted_columns = [
        'critic_name',
        'rotten_tomatoes_link',
        'movie_title',
        'content_rating',
        'genres',
        'original_release_date',
        'review_content',
        'review_type',
        'movie_info'
    ]

    #remove empty rows
    df_fixed = df.dropna(subset=wanted_columns)

    # take wanted columns. Use .copy() to avoid SettingWithCopyWarning
    df_filtered = df_fixed[wanted_columns].copy()

    #lowercase dataset
    for col in df_filtered.select_dtypes(include=['object']).columns:
        df_filtered.loc[:, col] = df_filtered[col].str.lower()

    #only take year from release date
    df_filtered['original_release_date'] = df_filtered['original_release_date'].str.split('-').str[0]

    return df_filtered

Save the dataset as a csv file.

In [4]:
def create_csv(df: pd.DataFrame):
    df.to_csv('../datasets/final_dataset.csv', index=False) 

Load data.

In [5]:
# Load datasets
df_critic_reviews = pd.read_csv('../datasets/rotten_tomatoes_critic_reviews.csv')
df_movies = pd.read_csv('../datasets/rotten_tomatoes_movies.csv')

print(f"Loaded {len(df_movies)} movies")
print(f"Loaded {len(df_critic_reviews)} reviews")

Loaded 17712 movies
Loaded 1130017 reviews


Run the code.

In [6]:
# Run the data preparation pipeline
merged = merge_datasets(df_movies, df_critic_reviews)
print(f"After merge: {len(merged)} rows")

df_final = fix_filter_dataset(merged)
print(f"After cleaning: {len(df_final)} rows")
print(f"Unique movies: {df_final['rotten_tomatoes_link'].nunique()}")


After merge: 1129887 rows
After cleaning: 1024931 rows
Unique movies: 16355


## Optional: Create Sampled Dataset

For faster processing during development, you can create a smaller sampled dataset. This reduces processing time from ~45 minutes to ~5-10 minutes.

**Configuration:**
- `CREATE_SAMPLE = True`: Creates a sampled dataset
- `CREATE_SAMPLE = False`: Uses the full dataset
- `SAMPLE_SIZE = 2000`: Number of movies to include in sample
- `DEMO_MOVIES`: Ensures specific movies are included for demonstrations

In [7]:
# Configuration
CREATE_SAMPLE = False  # Set to False for full dataset
SAMPLE_SIZE = 2000     # Number of movies to include

# Movies to ensure are in the sample (for demonstrations)
DEMO_MOVIES = ['aliens', 'toy story', 'the godfather']

if CREATE_SAMPLE:
    print(f"Creating sampled dataset with {SAMPLE_SIZE} movies...")
    
    # Group by movie to get unique movies
    movie_groups = df_final.groupby('rotten_tomatoes_link')
    unique_movies = list(movie_groups.groups.keys())
    
    print(f"Total unique movies: {len(unique_movies)}")
    
    # Find demo movies
    demo_links = []
    for link in unique_movies:
        movie_title = movie_groups.get_group(link)['movie_title'].iloc[0]
        if movie_title in DEMO_MOVIES:
            demo_links.append(link)
            print(f"  Found demo movie: {movie_title}")
    
    # Separate demo movies from others
    other_links = [link for link in unique_movies if link not in demo_links]
    
    # Sample from other movies
    import random
    random.seed(42)
    remaining_sample_size = SAMPLE_SIZE - len(demo_links)
    sampled_links = random.sample(other_links, min(remaining_sample_size, len(other_links)))
    
    # Combine demo movies with sampled movies
    final_links = demo_links + sampled_links
    
    # Filter dataframe to include only selected movies
    df_final = df_final[df_final['rotten_tomatoes_link'].isin(final_links)]
    
    print(f"\nSampled dataset created:")
    print(f"  Total movies: {df_final['rotten_tomatoes_link'].nunique()}")
    print(f"  Total reviews: {len(df_final)}")
    print(f"  Demo movies included: {len(demo_links)}")
else:
    print(f"Using full dataset: {df_final['rotten_tomatoes_link'].nunique()} movies")


Using full dataset: 16355 movies


In [8]:
# Save the final dataset
create_csv(df_final)
print("Final dataset saved to ../datasets/final_dataset.csv")


Final dataset saved to ../datasets/final_dataset.csv


Verification

In [9]:
# Quick verification
print(f"\nFinal dataset: {len(df_final)} rows, {df_final.shape[1]} columns")
print(f"Unique movies: {df_final['rotten_tomatoes_link'].nunique()}")
df_final.head()



Final dataset: 1024931 rows, 9 columns
Unique movies: 16355


,critic_name,rotten_tomatoes_link,movie_title,content_rating,genres,original_release_date,review_content,review_type,movie_info
0,andrew l. urban,m/0814255,percy jackson & the olympians: the lightning t...,pg,"action & adventure, comedy, drama, science fic...",2010,a fantasy adventure that fuses greek mythology...,fresh,"always trouble-prone, the life of teenager per..."
1,louise keller,m/0814255,percy jackson & the olympians: the lightning t...,pg,"action & adventure, comedy, drama, science fic...",2010,"uma thurman as medusa, the gorgon with a coiff...",fresh,"always trouble-prone, the life of teenager per..."
3,ben mceachen,m/0814255,percy jackson & the olympians: the lightning t...,pg,"action & adventure, comedy, drama, science fic...",2010,whether audiences will get behind the lightnin...,fresh,"always trouble-prone, the life of teenager per..."
4,ethan alter,m/0814255,percy jackson & the olympians: the lightning t...,pg,"action & adventure, comedy, drama, science fic...",2010,what's really lacking in the lightning thief i...,rotten,"always trouble-prone, the life of teenager per..."
5,david germain,m/0814255,percy jackson & the olympians: the lightning t...,pg,"action & adventure, comedy, drama, science fic...",2010,it's more a list of ingredients than a movie-m...,rotten,"always trouble-prone, the life of teenager per..."
